In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
csv_source_file_path = 'csv_source_files/'
csv_result_file_path = 'csv_result_files/'

#load cases
source_cases = pd.read_csv(csv_source_file_path + '455fd63b-603d-4608-8216-7d8647f43350.csv')
source_cases = source_cases.dropna(subset=['Accurate_Episode_Date','Case_Reported_Date','Test_Reported_Date','Specimen_Date'])
source_cases = source_cases[(source_cases['Reporting_PHU_City']=='Toronto') | 
                            (source_cases['Reporting_PHU_City']=='Ottawa') |
                            (source_cases['Reporting_PHU_City']=='Whitby') |
                            (source_cases['Reporting_PHU_City']=='Oakville') |
                            (source_cases['Reporting_PHU_City']=='Mississauga') |
                            (source_cases['Reporting_PHU_City']=='Newmarket')]

source_cases['Case_Reported_Date'] = pd.to_datetime(source_cases['Case_Reported_Date']).dt.strftime('%Y-%m-%d')
source_cases = source_cases[ (source_cases['Case_Reported_Date']>'2020-10-31') & (source_cases['Case_Reported_Date']<'2021-03-01') ]

source_cases['Accurate_Episode_Date'] = pd.to_datetime(source_cases['Accurate_Episode_Date']).dt.strftime('%Y-%m-%d')
source_cases['Test_Reported_Date'] = pd.to_datetime(source_cases['Test_Reported_Date']).dt.strftime('%Y-%m-%d')
source_cases['Specimen_Date'] = pd.to_datetime(source_cases['Specimen_Date']).dt.strftime('%Y-%m-%d')

#load pre-processed dimensions
Date_dimension = pd.read_csv(csv_result_file_path + "Date_dimension.csv")
Mobility_dimension = pd.read_csv(csv_result_file_path + "pre_Mobility_dimension.csv")
Weather_dimension = pd.read_csv(csv_result_file_path + "pre_Weather_dimension.csv")
PHU_Location_dimension = pd.read_csv(csv_result_file_path + "PHU_Location_dimension.csv")
Patient_dimension = pd.read_csv(csv_result_file_path + "Patient_dimension.csv")
Special_measures_dimension = pd.read_csv(csv_result_file_path + "pre_Special_Measures_dimension.csv")



In [3]:
fact_table = pd.DataFrame(columns=['Onset_date_key','Reported_date_key','Test_date_key','Specimen_date_key',
                                   'Patient_key','PHU_Location_key','Mobility_key','Special_measures_key','Weather_key',
                                   'Resolved','Unresolved','Fatal'])

In [4]:

current_row = 0
row_length = len(source_cases)
start = time.time()

for idx, row in source_cases.iterrows():
    current_row += 1
    if(current_row%5000 == 0):
        percentage = round((current_row/row_length)*100, 2)
        seconds_spent = time.time()-start
        remaining = seconds_spent/percentage*100/60
        print("%s of %s      %s percent done\nseconds spent: %s      remaining minutes: %s\n"%
              (str(current_row), str(row_length), str(percentage),str(seconds_spent),str(remaining)))
    
    
    resolved = False
    unresolved = False
    fatal = False
    if (row["Outcome1"] == "Resolved"):
        resolved = True
    elif (row["Outcome1"] == "Not Resolved"):
        unresolved = True
    elif(row["Outcome1"] == "Fatal"):
        fatal = True
    
    use_weather_of = 'Toronto'
    if row["Reporting_PHU_City"] == 'Ottawa':
        use_weather_of = 'Ottawa'
    
    fact_table_row = [Date_dimension[ row["Accurate_Episode_Date"]==Date_dimension['date'] ]['Date_surrogate_key'].values[0],
                      Date_dimension[ row["Case_Reported_Date"]==Date_dimension['date'] ]['Date_surrogate_key'].values[0],
                      Date_dimension[ row["Test_Reported_Date"]==Date_dimension['date'] ]['Date_surrogate_key'].values[0],
                      Date_dimension[ row["Specimen_Date"]==Date_dimension['date'] ]['Date_surrogate_key'].values[0],
                      current_row - 1, #this is the Patient_key
                      PHU_Location_dimension[PHU_Location_dimension["ID"]==row["Reporting_PHU_ID"]]['PHU_Location_key'].values[0],
                      Mobility_dimension[ (row["Reporting_PHU_City"]==Mobility_dimension["Subregion"])&
                                         (row["Case_Reported_Date"]==Mobility_dimension["date"]) ]['Mobility_key'].values[0],
                      Special_measures_dimension[ (row["Reporting_PHU_ID"]==Special_measures_dimension["Reporting_PHU_id"])&
                                                 (row["Case_Reported_Date"]>=Special_measures_dimension["Start_date"]) & 
                                                 (row["Case_Reported_Date"]<=Special_measures_dimension["End_date"])
                                                ]['Special_Measures_key'].values[0],
                      Weather_dimension[ (Weather_dimension["Station Name"]==use_weather_of)&
                                        (row["Case_Reported_Date"]==Weather_dimension["date"]) ].values[0],
                      resolved,unresolved,fatal]
    
    
    
    fact_table.loc[len(fact_table)] = fact_table_row
print("Finished with %s minutes"%((time.time()-start)/60))

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-16
2020-12-25

2020-12-15
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-12-14
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-12-14
2020-12-25

2020-12-17
2020-11-16
2020-12-25

2020-12-17
202

2020-11-27
2020-11-07
2020-12-25

2020-11-27
2020-11-07
2020-12-25

2020-11-27
2020-11-07
2020-12-25

2020-11-27
2020-11-07
2020-12-25

2020-11-27
2020-11-07
2020-12-25

2020-11-27
2020-11-07
2020-12-25

2020-11-29
2020-11-07
2020-12-25

2020-11-29
2020-11-07
2020-12-25

2020-11-19
2020-11-07
2020-12-25

2020-11-19
2020-11-07
2020-12-25

2020-11-19
2020-11-07
2020-12-25

2020-11-19
2020-11-07
2020-12-25

2020-11-19
2020-11-07
2020-12-25

2020-11-19
2020-11-07
2020-12-25

2020-11-19
2020-11-07
2020-12-25

2020-11-19
2020-11-07
2020-12-25

2020-11-19
2020-11-07
2020-12-25

2020-11-19
2020-11-07
2020-12-25

2020-11-19
2020-11-07
2020-12-25

2020-11-19
2020-11-07
2020-12-25

2020-11-19
2020-11-07
2020-12-25

2020-11-20
2020-11-07
2020-12-25

2020-11-20
2020-11-07
2020-12-25

2020-11-20
2020-11-07
2020-12-25

2020-11-20
2020-11-07
2020-12-25

2020-11-20
2020-11-07
2020-12-25

2020-11-24
2020-11-07
2020-12-25

2020-11-24
2020-11-07
2020-12-25

2020-11-24
2020-11-07
2020-12-25

2020-11-24
202

2020-12-17
2020-11-16
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-19
2020-11-07
2020-12-25

2020-12-19
2020-11-07
2020-12-25

2020-12-19
2020-11-07
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
202

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-19
2020-11-23
2020-12-25

2020-12-19
2020-11-23
2020-12-25

2020-12-18
2020-12-14
2020-12-25

2020-12-28
2020-12-26
2021-02-15

2020-12-19
2020-11-23
2020-12-25

2020-12-24
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-16
2020-11-23
2020-12-25

2020-12-16
2020-11-23
2020-12-25

2020-12-16
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-18
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-17
2020-11-23
2020-12-25

2020-12-19
202

2020-12-18
2020-11-23
2020-12-25

2020-12-19
2020-12-14
2020-12-25

2020-12-19
2020-12-14
2020-12-25

2020-12-19
2020-12-14
2020-12-25

2020-12-19
2020-12-14
2020-12-25

2020-12-19
2020-12-14
2020-12-25

2020-12-19
2020-12-14
2020-12-25

2020-12-19
2020-11-23
2020-12-25

2020-12-19
2020-11-23
2020-12-25

Finished with 0.14324821631113688 minutes


In [5]:
fact_table.head()

,Onset_date_key,Reported_date_key,Test_date_key,Specimen_date_key,Patient_key,PHU_Location_key,Mobility_key,Special_measures_key,Weather_key,Resolved,Unresolved,Fatal
0,715,716,716,715,0,2,1461,97,"[166, 6158731, -3.0, -8.0, -5.0, 1.0, 0.0, 1.0...",True,False,False
1,714,716,716,714,1,2,1461,97,"[166, 6158731, -3.0, -8.0, -5.0, 1.0, 0.0, 1.0...",True,False,False
2,710,716,716,715,2,2,1461,97,"[166, 6158731, -3.0, -8.0, -5.0, 1.0, 0.0, 1.0...",True,False,False
3,709,716,716,714,3,2,1461,97,"[166, 6158731, -3.0, -8.0, -5.0, 1.0, 0.0, 1.0...",True,False,False
4,714,716,716,715,4,1,1076,41,"[166, 6158731, -3.0, -8.0, -5.0, 1.0, 0.0, 1.0...",True,False,False


In [6]:
csv_result_file_path = 'csv_result_files/'
fact_table.to_csv(csv_result_file_path + 'Fact_table.csv',index=False)